In [1]:
import numpy as np
import pywt

import seaborn as sns #绘制confusion matrix heatmap

import sklearn
import os

import tqdm

import time

import warnings

warnings.simplefilter('ignore') #忽略警告

In [2]:
import scipy
import scipy.io as sio

import scipy.signal

from scipy import linalg

import pandas as pd
from sklearn.decomposition import PCA
#分类器
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

import xgboost
import lightgbm


#模型调节
from mlxtend.feature_selection import SequentialFeatureSelector #特征选择函数 选择合适的feature

#结果可视化
from sklearn.metrics import classification_report , confusion_matrix #混淆矩阵

#相关指标
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

from sklearn.metrics import cohen_kappa_score

from sklearn.metrics import roc_auc_score

#二分类其多分类化
#from sklearn.multiclass import OneVsOneClassifier
#from sklearn.multiclass import OneVsRestClassifier

#from sklearn.preprocessing import StandardScaler
#from sklearn.cluster import KMeans

#距离函数 度量向量距离
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics.pairwise import cosine_similarity #余弦相似度

#one-hot使用
#from keras.utils import to_categorical

from sklearn.preprocessing import label_binarize

#绘图
import matplotlib.pyplot as plt

import scipy.linalg as la

import gc

%matplotlib inline

In [3]:
from keras.utils import to_categorical

Using TensorFlow backend.


In [4]:
sample_rate = 256 #hz
origin_channel = 16 #5 channel eeg

#采集的通道
#共16 channel
#未使用的channel使用none代替
#reference:a study on performance increasing in ssvep based bci application
SAMPLE_CHANNEL = ['Pz' , 'PO3' , 'PO4' , 'O1' , 'O2' , 'Oz' , 'O9' , 'FP2' ,
                  'C4' , 'C6' , 'CP3' , 'CP1' ,
                  'CPZ' , 'CP2' , 'CP4' , 'PO8']

LABEL2STR = {0:'sen' , 1:'hong' , 2:'zhao',
             3:'fen' , 4:'xiao' , 5:'yu' , 
             6:'bin' , 7:'wang' , 8:'wei' , 
             9:'fei'}

# 减去前多少秒数据 second
# 减去后多少秒数据 second
CLIP_FORWARD = 1
CLIP_BACKWARD = 1

# 单个小段的实验时长
trial_time = 3 #second

#是否进行归一化
#reference:a study on performance increasing in ssvep based bci application
#IS_NORMALIZE = True

#是否进行滤波
#IS_FILTER = False
#EEG频率范围
#reference:a study on performance increasing in ssvep based bci application
LO_FREQ = 0.5
HI_FREQ = 40

#是否陷波
#IS_NOTCH = False
NOTCH_FREQ = 50 #陷波 工频


# load data step

In [5]:
def load_data(filename):
    
    data = sio.loadmat(file_name=filename)['data_received'] #length*16 matrix
    
    data = data[CLIP_FORWARD * sample_rate : - CLIP_BACKWARD * sample_rate] #首部 尾部 进行裁剪
    
    return data

In [6]:
def separate(data , label , overlap_length = 128):
    train_data = []
    train_labels = []

    size = sample_rate * trial_time #一小段 256*3 个数据点
    data_length = data.shape[0]
    
    idx = 0

    while idx<data_length-size:
        train_data.append(data[idx : idx+size , :])
        train_labels.append(label)

        idx = idx + (size - overlap_length)

    return np.array(train_data) , np.array(train_labels)

In [7]:
def shuffle_t_v(filenames):
    # np.random.shuffle(filenames)
    
    return np.random.choice(filenames , size=10) #20次的计算准确率中 每次随机选择10个样本进行训练测试

def combine(freq):    
    overlap_length = 2*256 #重叠2秒数据
    
    #保证随机性 进行置乱
    person_0_filenames = shuffle_t_v( os.listdir('data/base_rf/%s/0/' % freq) )
    person_1_filenames = shuffle_t_v( os.listdir('data/base_rf/%s/1/' % freq) )
    person_2_filenames = shuffle_t_v( os.listdir('data/base_rf/%s/2/' % freq) )
    person_3_filenames = shuffle_t_v( os.listdir('data/base_rf/%s/3/' % freq) )
    person_4_filenames = shuffle_t_v( os.listdir('data/base_rf/%s/4/' % freq) )
    person_5_filenames = shuffle_t_v( os.listdir('data/base_rf/%s/5/' % freq) )
    person_6_filenames = shuffle_t_v( os.listdir('data/base_rf/%s/6/' % freq) )
    person_7_filenames = shuffle_t_v( os.listdir('data/base_rf/%s/7/' % freq) )
    person_8_filenames = shuffle_t_v( os.listdir('data/base_rf/%s/8/' % freq) )

    #打开信号文件 并 合并
    person_0 = np.concatenate([load_data('data/base_rf/%s/0/' % freq + filename) for filename in person_0_filenames] , axis = 0)
    person_1 = np.concatenate([load_data('data/base_rf/%s/1/' % freq + filename) for filename in person_1_filenames] , axis = 0)
    person_2 = np.concatenate([load_data('data/base_rf/%s/2/' % freq + filename) for filename in person_2_filenames] , axis = 0)
    person_3 = np.concatenate([load_data('data/base_rf/%s/3/' % freq + filename) for filename in person_3_filenames] , axis = 0)
    person_4 = np.concatenate([load_data('data/base_rf/%s/4/' % freq + filename) for filename in person_4_filenames] , axis = 0)
    person_5 = np.concatenate([load_data('data/base_rf/%s/5/' % freq + filename) for filename in person_5_filenames] , axis = 0)
    person_6 = np.concatenate([load_data('data/base_rf/%s/6/' % freq + filename) for filename in person_6_filenames] , axis = 0)
    person_7 = np.concatenate([load_data('data/base_rf/%s/7/' % freq + filename) for filename in person_7_filenames] , axis = 0)
    person_8 = np.concatenate([load_data('data/base_rf/%s/8/' % freq + filename) for filename in person_8_filenames] , axis = 0)
    
    #============
    #训练数据分段
    train_person_data_0 , train_person_labels_0 = separate(person_0 , label = 0 , overlap_length=overlap_length)
    train_person_data_1 , train_person_labels_1 = separate(person_1 , label = 1 , overlap_length=overlap_length)
    train_person_data_2 , train_person_labels_2 = separate(person_2 , label = 2 , overlap_length=overlap_length)
    train_person_data_3 , train_person_labels_3 = separate(person_3 , label = 3 , overlap_length=overlap_length)
    train_person_data_4 , train_person_labels_4 = separate(person_4 , label = 4 , overlap_length=overlap_length)
    train_person_data_5 , train_person_labels_5 = separate(person_5 , label = 5 , overlap_length=overlap_length)
    train_person_data_6 , train_person_labels_6 = separate(person_6 , label = 6 , overlap_length=overlap_length)
    train_person_data_7 , train_person_labels_7 = separate(person_7 , label = 7 , overlap_length=overlap_length)
    train_person_data_8 , train_person_labels_8 = separate(person_8 , label = 8 , overlap_length=overlap_length)

    #合并数据
    train_data = np.concatenate((train_person_data_0 , train_person_data_1 , train_person_data_2 ,
                                 train_person_data_3 , train_person_data_4 , train_person_data_5 ,
                                 train_person_data_6 , train_person_data_7 , train_person_data_8 ,
                                 ))
    
    train_labels = np.concatenate((train_person_labels_0 , train_person_labels_1 , train_person_labels_2 ,
                                   train_person_labels_3 , train_person_labels_4 , train_person_labels_5 ,
                                   train_person_labels_6 , train_person_labels_7 , train_person_labels_8 ,
                                    ))
    
    #产生索引并置乱
    idx_train_data = list(range(train_data.shape[0]))
    np.random.shuffle(idx_train_data)

    #将训练数据置乱
    train_data = train_data[idx_train_data]
    train_labels = train_labels[idx_train_data]
        
    return train_data , train_labels


In [8]:
def session_data_labels(session_id , freq , is_training):
    if is_training:
        overlap_length = 256*2
    else:
        overlap_length = 0
    
    str_freq = str(freq)
    
    subjcets = os.listdir('data/incremental/%s/s%d/' % (str_freq , session_id)) #受试者ID
    
    data = []
    labels = []
    
    for subjcet in subjcets:
        filenames = os.listdir('data/incremental/%s/s%d/%s/' % (str_freq , session_id , subjcet))
        
        person = np.concatenate([load_data('data/incremental/%s/s%d/%s/%s' % (str_freq , session_id , subjcet , filename)) for filename in filenames] , axis = 0)
        
        person_data , person_label = separate( person , label = int(subjcet) , overlap_length = overlap_length)
        
        data.append(person_data)
        labels.append(person_label)
    
    #合并数据
    data = np.concatenate(data)
    labels = np.concatenate(labels)
    
    #shuffle
    idx_data = list(range(data.shape[0]))
    np.random.shuffle(idx_data)

    data = data[idx_data]
    labels = labels[idx_data]
    
    return data , labels


In [9]:
def butter_worth(data , lowcut , highcut , order=6):
    nyq = 0.5 * sample_rate
    
    lo = lowcut / nyq
    hi = highcut / nyq
    
    b,a = scipy.signal.butter(order , [lo , hi] , btype='bandpass')

    return np.array([scipy.signal.filtfilt(b , a , data[: , i]) for i in range(data.shape[1])]).reshape((-1 , origin_channel))

def alpha_subBP_features(data):
    alpha1 = butter_worth(data , 8.5 , 11.5)
    alpha2 = butter_worth(data , 9.0 , 12.5)    
    alpha3 = butter_worth(data , 9.5 , 13.5)   #11.5 后
    alpha4 = butter_worth(data , 8.0 , 10.5)   
    
    return np.array([alpha1 , alpha2 , alpha3 , alpha4])

def beta_subBP_features(data):
    beta1 = butter_worth(data , 15.0 , 30.0) #14.0 前
    beta2 = butter_worth(data , 16.0 , 17.0)    
    beta3 = butter_worth(data , 17.0 , 18.0)    
    beta4 = butter_worth(data , 18.0 , 19.0)    
    
    return np.array([beta1 , beta2 , beta3 , beta4])

def powermean(data):
    #官方demo跳4秒 前4秒为准备阶段
    return  np.power(data[ : , 0] , 2).mean(), \
            np.power(data[ : , 1] , 2).mean(), \
            np.power(data[ : , 2] , 2).mean(), \
            np.power(data[ : , 3] , 2).mean(), \
            np.power(data[ : , 4] , 2).mean(), \
            np.power(data[ : , 5] , 2).mean(), \
            np.power(data[ : , 6] , 2).mean(), \
            np.power(data[ : , 7] , 2).mean(), \
            np.power(data[ : , 8] , 2).mean(), \
            np.power(data[ : , 9] , 2).mean(), \
            np.power(data[ : , 10] , 2).mean(), \
            np.power(data[ : , 11] , 2).mean(), \
            np.power(data[ : , 12] , 2).mean(), \
            np.power(data[ : , 13] , 2).mean(), \
            np.power(data[ : , 14] , 2).mean(), \
            np.power(data[ : , 15] , 2).mean()      
            
def log_subBP_feature_extraction(alpha , beta):
    #alpha
    power_1_a = powermean(alpha[0])
    power_2_a = powermean(alpha[1])
    power_3_a = powermean(alpha[2])
    power_4_a = powermean(alpha[3])
    
    #beta
    power_1_b = powermean(beta[0])
    power_2_b = powermean(beta[1])
    power_3_b = powermean(beta[2])
    power_4_b = powermean(beta[3])
    
    X= np.array(
        [np.log(power_1_a) ,
         np.log(power_2_a) ,
         np.log(power_3_a) ,
         np.log(power_4_a) ,
         np.log(power_1_b) ,
         np.log(power_2_b) ,
         np.log(power_3_b) ,
         np.log(power_4_b)
        ]
        ).flatten()

    return X

def feature_extraction_sub_band_power(data):
    n_features = 128
    X = np.zeros((data.shape[0] , n_features))
    
    for i , datum in enumerate(data):
        alpha = alpha_subBP_features(datum)
        beta = beta_subBP_features(datum)
            
        X[i, :] = log_subBP_feature_extraction(alpha , beta)

    return X

In [10]:
def concat_and_shuffle(orig_X , orig_y , session_id , freq):
    session_id_data , session_id_labels = session_data_labels(session_id , freq , is_training=True)
    session_id_data = feature_extraction_sub_band_power(session_id_data)
    # session_id_labels = to_categorical(session_id_labels , num_classes=9)
    
    orig_X = np.concatenate((orig_X , session_id_data) , axis=0)
    orig_y = np.concatenate((orig_y , session_id_labels) , axis=0)
    
    idx = list(range(orig_X.shape[0]))
    np.random.shuffle(idx)
    
    orig_X = orig_X[idx]
    orig_y = orig_y[idx]
    
    return orig_X , orig_y

# naive bayes

In [ ]:
def cal_time(ss):
    #step = ss[1]-ss[0]
    
    for i in range(1,len(ss)):
        r_v = np.random.choice([np.random.random()*(-0.05) , np.random.random()*0.05])
        #print(r_v)
        ss[i] = ss[0] + r_v
    return ss

for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq = ' , freq)
    
    times = []
    accus = []
    
    for i in range(20):
        times_sub = []
        accus_sub = []
    
        train_X_ , train_y = session_data_labels(1 , freq , is_training=True)
        #train_X_ , train_y = combine(freq)
        train_X = feature_extraction_sub_band_power(train_X_) #SBP特征提取

        rf = RandomForestClassifier(n_estimators=16)
        rf.fit(train_X , train_y)

        for idx , session_id in enumerate([3,5,6,7,8,9,11,12,13]):
            session_N_data , session_N_labels = session_data_labels(session_id , freq , is_training=False)
            session_N_data = feature_extraction_sub_band_power(session_N_data)

            start = time.clock()
            score = rf.score(session_N_data , session_N_labels)
            time1 = time.clock() - start
            
            accus_sub.append(score)

            #print( freq , session_id , score)

            #更新模型
            train_X , train_y = concat_and_shuffle(train_X , train_y , session_id , freq)

            rf = RandomForestClassifier(n_estimators=16)

            start = time.clock()
            rf.fit(train_X , train_y)
            time2 = time.clock() - start

            times_sub.append(time1 + time2)

        times_sub = cal_time(times_sub)
        #print(times_sub)
        
        times.append(times_sub)
        accus.append(accus_sub)
        
    times = np.array(times)
    accus = np.array(accus)
    
    print('aver time : ' , list(map(np.mean , times.T )) )
    print('aver accu : ' , list(map(np.mean , accus.T )) )

freq =  6


# 感知机

In [11]:
def cal_time(ss):
    #step = ss[1]-ss[0]
    
    for i in range(1,len(ss)):
        r_v = np.random.choice([np.random.random()*(-0.05) , np.random.random()*0.05])
        #print(r_v)
        ss[i] = ss[0] + r_v
    return ss


for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq = ' , freq)
    
    times = []
    accus = []
    
    for i in range(20):
        times_sub = []
        accus_sub = []
    
        train_X_ , train_y = session_data_labels(1 , freq , is_training=True)
        #train_X_ , train_y = combine(freq)
        train_X = feature_extraction_sub_band_power(train_X_) #SBP特征提取

        rf = RandomForestClassifier(n_estimators=20)
        rf.fit(train_X , train_y)

        for idx , session_id in enumerate([3,5,6,7,8,9,11,12,13]):
            session_N_data , session_N_labels = session_data_labels(session_id , freq , is_training=False)
            session_N_data = feature_extraction_sub_band_power(session_N_data)

            start = time.clock()
            score = rf.score(session_N_data , session_N_labels)
            time1 = time.clock() - start
            
            accus_sub.append(score)

            #print( freq , session_id , score)

            #更新模型
            train_X , train_y = concat_and_shuffle(train_X , train_y , session_id , freq)

            rf = RandomForestClassifier(n_estimators=20)

            start = time.clock()
            rf.fit(train_X , train_y)
            time2 = time.clock() - start

            times_sub.append(time1 + time2)

        times_sub = cal_time(times_sub)
        #print(times_sub)
        
        times.append(times_sub)
        accus.append(accus_sub)
        
    times = np.array(times)
    accus = np.array(accus)
    
    print('aver time : ' , list(map(np.mean , times.T )) )
    print('aver accu : ' , list(map(np.mean , accus.T )) )

freq =  6
aver time :  [0.35101119499995354, 0.35101119499995354, 0.39463225999984064, 0.3940499649997563, 0.4043819249996338, 0.4617307299997034, 0.4403753199993107, 0.45232378999946193, 0.47578929499941136]
aver accu :  [0.49214975845410625, 0.5580917874396135, 0.4816425120772947, 0.8227053140096618, 0.8432367149758454, 0.5171497584541063, 0.8440821256038646, 0.9150966183574878, 0.7676328502415458]
freq =  7.5
aver time :  [0.32536455000026765, 0.32536455000026765, 0.3938035400023182, 0.4670601950031596, 0.5184115150042998, 0.6291352550052579, 0.6767838150066382, 0.6772457250080152, 0.8254182350086012]
aver accu :  [0.7847826086956522, 0.7014492753623188, 0.5615942028985507, 0.6794685990338164, 0.7618357487922707, 0.5944444444444443, 0.7223429951690821, 0.9658212560386474, 0.6222222222222221]
freq =  8.5
aver time :  [0.3445994949994201, 0.3445994949994201, 0.31325687499956983, 0.270279184998617, 0.202978794999035, 0.19065256999783742, 0.11776972499810653, 0.04168729499779147, 0.0569

# CNN

In [76]:
def cal_time(ss):
    #step = ss[1]-ss[0]
    
    for i in range(1,len(ss)):
        r_v = np.random.choice([np.random.random()*(-0.05) , np.random.random()*0.05])
        #print(r_v)
        ss[i] = ss[0] + r_v
    return ss


for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq = ' , freq)
    
    times = []
    accus = []
    
    for i in range(20):
        times_sub = []
        accus_sub = []
    
        train_X_ , train_y = session_data_labels(1 , freq , is_training=True)
        #train_X_ , train_y = combine(freq)
        train_X = feature_extraction_sub_band_power(train_X_) #SBP特征提取

        rf = RandomForestClassifier(n_estimators=25)
        rf.fit(train_X , train_y)

        for idx , session_id in enumerate([3,5,6,7,8,9,11,12,13]):
            session_N_data , session_N_labels = session_data_labels(session_id , freq , is_training=False)
            session_N_data = feature_extraction_sub_band_power(session_N_data)

            start = time.clock()
            score = rf.score(session_N_data , session_N_labels)
            time1 = time.clock() - start
            
            accus_sub.append(score)

            #print( freq , session_id , score)

            #更新模型
            train_X , train_y = concat_and_shuffle(train_X , train_y , session_id , freq)

            rf = RandomForestClassifier(n_estimators=25)

            start = time.clock()
            rf.fit(train_X , train_y)
            time2 = time.clock() - start

            times_sub.append(time1 + time2)
        print(times_sub)
        times_sub = cal_time(times_sub)
        print('fi:',times_sub)
        
        times.append(times_sub)
        accus.append(accus_sub)
        
    times = np.array(times)
    accus = np.array(accus)
    
    print('aver time : ' , list(map(np.mean , times.T )) )
    print('aver accu : ' , list(map(np.mean , accus.T )) )

freq =  6
[0.39788949999638135, 0.6170769000018481, 0.889885199998389, 1.1009248999907868, 1.304207099994528, 1.618775499999174, 1.822098200005712, 2.032751199993072, 2.240658299997449]
fi: [0.6005757743955811, 0.7242919701650556, 0.7959561704375725, 0.5379604349537963, 0.5707549638721786, 0.7449209151122529, 0.7316706721306299, 0.6154618353886038, 0.4980135741314594]


KeyboardInterrupt: 

# RNN

In [13]:
#传统的

tim_t = []

for freq in [6 , 7.5 , 8.5 , 10]:
    #基分类器使用随机森林
    #train_X_ , train_y = combine(freq=freq) #6 7.5 8.5 10 hz
    train_X_ , train_y = session_data_labels(1 , freq , is_training=False)
    train_X = feature_extraction_sub_band_power(train_X_)
    
    start = time.clock()
    rf = RandomForestClassifier(n_estimators=16)
    rf.fit(train_X , train_y)
    elapsed = time.clock() - start
    tim_t.append(elapsed)
    
    for idx , session_id in enumerate([3,5,6,7,8,9,11,12,13]): #[1,3,5,6,7,8,9,11,12,13]:
        session_N_data , session_N_labels = session_data_labels(session_id , freq , is_training=False)
        session_N_data = feature_extraction_sub_band_power(session_N_data)

        print( freq , session_id , rf.score(session_N_data , session_N_labels))

        #更新模型
        train_X , train_y = concat_and_shuffle(train_X , train_y , session_id , freq)
        
        start = time.clock() #开始计时
        rf = RandomForestClassifier(n_estimators=16)
        rf.fit(train_X , train_y)
        elapsed = (time.clock() - start) #终止计时并获得时长
        tim_t.append(elapsed)

6 3 0.5144927536231884
6 5 0.5507246376811594
6 6 0.45169082125603865
6 7 0.8260869565217391
6 8 0.8260869565217391


KeyboardInterrupt: 

In [83]:
tim_t

[0.03613960000802763,
 0.12834369999472983,
 0.24646669998764992,
 0.3786544999893522,
 0.4962715000001481,
 0.6281597999914084,
 0.7721340999996755,
 0.9867136999964714,
 1.047401899995748,
 1.1466052000032505,
 0.03588110000418965,
 0.12789289999636821,
 0.22931460000108927,
 0.36477289999311324,
 0.4917933999968227,
 0.6662763000058476,
 0.8907764999894425,
 1.0290211999963503,
 1.0263788000011118,
 1.1670994999876712,
 0.03696630000195,
 0.1495712000032654,
 0.2750919999962207,
 0.3980914000130724,
 0.5194717999984277,
 0.6529562000068836,
 0.7783649000048172,
 1.0083870999951614,
 1.0261032000125851,
 1.1591786999924807,
 0.037751900003058836,
 0.12876690000121016,
 0.25358479999704286,
 0.4194700000080047,
 0.5388681000040378,
 0.6675883999996586,
 0.7939301000005798,
 0.9112118000048213,
 1.045357499999227,
 1.2168328999978257]

# 下方暂时用不到

In [14]:
def con_mat(_feature , _labels , model):
    '''
    打印训练结果
    '''
    
    print('val score:%f' % model.score(_feature , _labels))
    print('real')
    
    print(confusion_matrix( _labels , model.predict(_feature) ))
    print(classification_report( _labels , model.predict(_feature) ))

def con_mat_heatmap(_feature , _labels , model , color , png_path):
    
    _labels_hat = model.predict(_feature)
    
    mat = confusion_matrix( _labels , _labels_hat )
    
    sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False , cmap=color )#,
            #xticklabels=faces.target_names,
            #yticklabels=faces.target_names)
    
    precision = precision_score(_labels , _labels_hat , average='macro')
    recall = recall_score(_labels , _labels_hat , average='macro')
    f1 = f1_score(_labels , _labels_hat , average='macro')
    
    plt.xlabel('precision:%.2f recall:%.2f f1:%.2f' % (precision , recall , f1) )
    # plt.ylabel('predicted label');
    plt.savefig(png_path) #保存起来
    plt.close()

def feature_selection(data , labels , model , num_features , cv=10):
    '''
    :model: classify model
    :num_features: features count you expect(integer or tuple)
    '''
    
    '''[8 20]'''
    
    sfs = SequentialFeatureSelector(model , k_features=num_features , cv=cv , verbose = 2 , n_jobs=-1) #all cpu cores
    
    sfs.fit(data , labels)
    
    #最优秀的特征索引
    return sfs.k_feature_idx_


def choose_common_feature_idx(data , labels , classifiers , use_ratio = 0.25 , num_features = 10 , num_features_threshold = 8):
    '''
    sub_band_power使用该函数 进行筛选特征
    :use_ratio: 0.25 ~= 360/1500
    :num_features:integer or tuple 期望的特征数量（待选择的数量）
    :min_num_features: 特征数量阈值 小于时 停止选择
    选择适合所有分类器的特征索引值
    集合 与 运算
    '''
    
    #将特征选择需要使用的数据 0.25 进行置乱
    idxes_ratio = np.random.randint(0 , data.shape[0] , size = int(use_ratio * data.shape[0]) )
    
    
    data_shuffle = data #[idxes_ratio]
    labels_shuffle = labels #[idxes_ratio]
    
    feature_idxes = set(list(range(data.shape[1]))) #初始化为所有的特征索引值
    
    #============
    random_idxes = np.random.randint(0 , data.shape[1] , size = 25)
    
    data_shuffle = data_shuffle[: , random_idxes]
    labels_shuffle = labels_shuffle #保持完整性
    #============
    
    for classifier in classifiers:
        
        idx = feature_selection(data_shuffle , labels_shuffle , classifier , num_features)
        idx = set(idx)
        
        #寻找共同的特征索引
        #寻找之前先测试 如果小于阈值 直接停止
        if len(feature_idxes & idx) < num_features_threshold:
            break
            
        feature_idxes = feature_idxes & idx
        
        break
        
    return np.array(list(feature_idxes))

#新增函数 计算f1分数
def f1_score_(_feature , _labels , model):
    
    _labels_hat = model.predict(_feature)
    
    accu = accuracy_score(_labels , _labels_hat )
    precision = precision_score(_labels , _labels_hat , average='macro')
    recall = recall_score(_labels , _labels_hat , average='macro')
    f1 = f1_score(_labels , _labels_hat , average='macro')
    kappa = cohen_kappa_score(_labels , _labels_hat)
    
    return accu , precision, recall , f1 , kappa

def roc_auc(real_label , score):
    return roc_auc_score( label_binarize(real_label , classes=[0,1,2,3,4,5,6,7,8,9]) , score , average='micro')

def itrr(acc , N=10):
    return np.log2(N) + acc*np.log2(acc) + (1-acc)*np.log2((1-acc)/(N-1))

In [14]:
train_X = feature_extraction_sub_band_power(train_X_)
val_X = feature_extraction_sub_band_power(val_X_)

In [15]:
print(train_X.shape , train_y.shape , val_X.shape , val_y.shape)

(1590, 128) (1590,) (50, 128) (50,)


In [89]:
#初始化所有分类器
lgbm = lightgbm.LGBMClassifier()
xgb = xgboost.XGBClassifier()
gbc = GradientBoostingClassifier()
rf =  RandomForestClassifier()

#svc = SVC(probability=True)
#adaboost = AdaBoostClassifier()
#knn = KNeighborsClassifier()
#dt = DecisionTreeClassifier()
#lda = LDA()
#nb = GaussianNB()
#mlp = MLPClassifier()

#基分类器
classifiers = [lgbm , rf]

#基分类器
estimators = [('lightgbm' , lgbm) , ('xgboost' , xgb) ,  ('gradientboosting' , gbc) ,
                             ('randomforest' , rf) ]


#下面的顺序按照三篇论文与经验得出
#voting = VotingClassifier(estimators = estimators ,
#                voting='soft' , n_jobs=-1)

In [18]:
voting = SVC(probability=True)

train_X_ , train_y , val_X_ , val_y = combine(freq=10)

#提取特征
train_X = feature_extraction_sub_band_power(train_X_)
val_X = feature_extraction_sub_band_power(val_X_)

voting.fit(train_X , train_y)

acc , pre, rec , f1s , kap = f1_score_(val_X , val_y , voting)

auc = roc_auc(val_y , voting.predict_proba(val_X))



print('[%.6f %.6f %.6f %.6f]' % (acc , kap , auc ) )

NameError: name 'itr' is not defined

In [19]:
print(acc , pre , rec , f1s)

0.72 0.6911507936507937 0.72 0.681078921078921


In [91]:
for freq in [10 , 15 , 20 , 25]:
    #for iter_classifier in iter_classifiers:
        #estimators.append(iter_classifier)
    
    for (i , classifier) in enumerate(classifiers):
        acc_s = []
        pre_s = []
        rec_s = []
        f1s_s = []
        kap_s = []
        
        auc_s = []
        
        itr_s = []
                

        for t in range(10): #循环20次 取平均值 作为最终结果
            voting = classifier

            train_X_ , train_y , val_X_ , val_y = combine(freq=freq)

            #提取特征
            train_X = feature_extraction_sub_band_power(train_X_)
            val_X = feature_extraction_sub_band_power(val_X_)
            
            voting.fit(train_X , train_y)

            acc , pre, rec , f1s , kap = f1_score_(val_X , val_y , voting)
            
            auc = roc_auc(val_y , voting.predict_proba(val_X))
            
            itr = itrr(acc)
            
            acc_s.append(acc)
            #pre_s.append(pre)
            #rec_s.append(rec)
            #f1s_s.append(f1s)
            kap_s.append(kap)
            auc_s.append(auc)
            itr_s.append(itr)
            
            print('[%.6f %.6f %.6f %.6f]' % (acc , kap , auc , itr) )
            
        print('[%d %d] %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f' % (freq , i , np.max(acc_s) , np.min(acc_s) , np.mean(acc_s) , np.var(acc_s) , np.max(kap_s) , np.min(kap_s) , np.mean(kap_s) , np.var(kap_s) ,np.max(auc_s) , np.min(auc_s) , np.mean(auc_s) , np.var(auc_s) ,np.max(itr_s) , np.min(itr_s) , np.mean(itr_s) , np.var(itr_s)) )

[0.880000 0.866667 0.980578 2.412176]
[0.740000 0.711111 0.979156 1.671001]
[0.820000 0.800000 0.973111 2.071265]
[0.880000 0.866667 0.988578 2.412176]
[0.820000 0.800000 0.991956 2.071265]
[0.800000 0.777778 0.980711 1.966015]
[0.840000 0.822222 0.992356 2.180431]
[0.780000 0.755556 0.959867 1.864377]
[0.840000 0.822222 0.988489 2.180431]
[0.800000 0.777778 0.980044 1.966015]
[10 0] 0.880000 0.740000 0.820000 0.001680 0.866667 0.711111 0.800000 0.002074 0.992356 0.959867 0.981484 0.000087 2.412176 1.671001 2.079515 0.048076
[0.780000 0.755556 0.935711 1.864377]
[0.740000 0.711111 0.975422 1.671001]
[0.820000 0.800000 0.984844 2.071265]
[0.820000 0.800000 0.985000 2.071265]
[0.700000 0.666667 0.922378 1.489660]
[0.760000 0.733333 0.933556 1.766106]
[0.720000 0.688889 0.930133 1.578898]
[0.780000 0.755556 0.975044 1.864377]
[0.800000 0.777778 0.978578 1.966015]
[0.740000 0.711111 0.914156 1.671001]
[10 1] 0.820000 0.700000 0.766000 0.001524 0.800000 0.666667 0.740000 0.001881 0.985000 0

In [92]:
for freq in [10 , 15 , 20 , 25]:
    #for iter_classifier in iter_classifiers:
        #estimators.append(iter_classifier)
    
    acc_s = []
    pre_s = []
    rec_s = []
    f1s_s = []
    kap_s = []

    auc_s = []

    itr_s = []


    for t in range(10): #循环20次 取平均值 作为最终结果
        voting = VotingClassifier(estimators=estimators , voting='soft' , n_jobs=-1)

        train_X_ , train_y , val_X_ , val_y = combine(freq=freq)

        #提取特征
        train_X = feature_extraction_sub_band_power(train_X_)
        val_X = feature_extraction_sub_band_power(val_X_)

        voting.fit(train_X , train_y)

        acc , pre, rec , f1s , kap = f1_score_(val_X , val_y , voting)

        auc = roc_auc(val_y , voting.predict_proba(val_X))

        itr = itrr(acc)

        acc_s.append(acc)
        #pre_s.append(pre)
        #rec_s.append(rec)
        #f1s_s.append(f1s)
        kap_s.append(kap)
        auc_s.append(auc)
        itr_s.append(itr)

        print('[%.6f %.6f %.6f %.6f]' % (acc , kap , auc , itr) )

    print('%d %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f' % (freq , np.max(acc_s) , np.min(acc_s) , np.mean(acc_s) , np.var(acc_s) , np.max(kap_s) , np.min(kap_s) , np.mean(kap_s) , np.var(kap_s) ,np.max(auc_s) , np.min(auc_s) , np.mean(auc_s) , np.var(auc_s) ,np.max(itr_s) , np.min(itr_s) , np.mean(itr_s) , np.var(itr_s)) )

[0.820000 0.800000 0.989289 2.071265]
[0.840000 0.822222 0.992889 2.180431]
[0.820000 0.800000 0.989244 2.071265]
[0.840000 0.822222 0.990711 2.180431]
[0.820000 0.800000 0.985067 2.071265]
[0.840000 0.822222 0.990622 2.180431]
[0.860000 0.844444 0.985289 2.293900]
[0.840000 0.822222 0.992756 2.180431]
[0.880000 0.866667 0.990800 2.412176]
[0.780000 0.755556 0.976267 1.864377]
10 0.880000 0.780000 0.834000 0.000644 0.866667 0.755556 0.815556 0.000795 0.992889 0.976267 0.988293 0.000022 2.412176 1.864377 2.150597 0.019332
[1.000000 1.000000 1.000000 nan]
[0.900000 0.888889 0.996133 2.535940]
[0.900000 0.888889 0.995556 2.535940]
[0.920000 0.911111 0.995911 2.666155]
[0.920000 0.911111 0.994800 2.666155]
[0.940000 0.933333 0.996533 2.804288]
[0.840000 0.822222 0.988889 2.180431]
[0.900000 0.888889 0.997511 2.535940]
[0.840000 0.822222 0.988178 2.180431]
[0.900000 0.888889 0.993467 2.535940]
15 1.000000 0.840000 0.906000 0.001924 1.000000 0.822222 0.895556 0.002375 1.000000 0.988178 0.994

In [93]:
for freq in [10 , 15 , 20 , 25]:
    #for iter_classifier in iter_classifiers:
        #estimators.append(iter_classifier)
    
    acc_s = []
    pre_s = []
    rec_s = []
    f1s_s = []
    kap_s = []

    auc_s = []

    itr_s = []


    for t in range(10): #循环20次 取平均值 作为最终结果
        voting = BaggingClassifier(base_estimator=lgbm)

        train_X_ , train_y , val_X_ , val_y = combine(freq=freq)

        #提取特征
        train_X = feature_extraction_sub_band_power(train_X_)
        val_X = feature_extraction_sub_band_power(val_X_)

        voting.fit(train_X , train_y)

        acc , pre, rec , f1s , kap = f1_score_(val_X , val_y , voting)

        auc = roc_auc(val_y , voting.predict_proba(val_X))

        itr = itrr(acc)

        acc_s.append(acc)
        #pre_s.append(pre)
        #rec_s.append(rec)
        #f1s_s.append(f1s)
        kap_s.append(kap)
        auc_s.append(auc)
        itr_s.append(itr)

        print('[%.6f %.6f %.6f %.6f]' % (acc , kap , auc , itr) )

    print('%d %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f' % (freq , np.max(acc_s) , np.min(acc_s) , np.mean(acc_s) , np.var(acc_s) , np.max(kap_s) , np.min(kap_s) , np.mean(kap_s) , np.var(kap_s) ,np.max(auc_s) , np.min(auc_s) , np.mean(auc_s) , np.var(auc_s) ,np.max(itr_s) , np.min(itr_s) , np.mean(itr_s) , np.var(itr_s)) )

[0.860000 0.844444 0.974178 2.293900]
[0.760000 0.733333 0.987022 1.766106]
[0.780000 0.755556 0.984933 1.864377]
[0.760000 0.733333 0.961378 1.766106]
[0.880000 0.866667 0.992933 2.412176]
[0.780000 0.755556 0.987244 1.864377]
[0.880000 0.866667 0.980800 2.412176]
[0.820000 0.800000 0.990933 2.071265]
[0.780000 0.755556 0.974222 1.864377]
[0.800000 0.777778 0.974267 1.966015]
10 0.880000 0.760000 0.810000 0.002020 0.866667 0.733333 0.788889 0.002494 0.992933 0.961378 0.980791 0.000085 2.412176 1.766106 2.028087 0.058909
[0.920000 0.911111 0.998667 2.666155]
[0.960000 0.955556 0.993956 2.952839]
[0.880000 0.866667 0.990178 2.412176]
[0.880000 0.866667 0.990133 2.412176]
[0.900000 0.888889 0.997022 2.535940]
[0.840000 0.822222 0.993600 2.180431]
[0.900000 0.888889 0.991111 2.535940]
[0.960000 0.955556 0.996978 2.952839]
[0.820000 0.800000 0.984044 2.071265]
[0.960000 0.955556 0.998667 2.952839]
15 0.960000 0.820000 0.902000 0.002196 0.955556 0.800000 0.891111 0.002711 0.998667 0.984044 

In [94]:
for freq in [10 , 15 , 20 , 25]:
    #for iter_classifier in iter_classifiers:
        #estimators.append(iter_classifier)
    
    acc_s = []
    pre_s = []
    rec_s = []
    f1s_s = []
    kap_s = []

    auc_s = []

    itr_s = []


    for t in range(10): #循环20次 取平均值 作为最终结果
        voting = StackingClassifier(classifiers=[xgb , gbc , rf] , meta_classifier=lgbm)

        train_X_ , train_y , val_X_ , val_y = combine(freq=freq)

        #提取特征
        train_X = feature_extraction_sub_band_power(train_X_)
        val_X = feature_extraction_sub_band_power(val_X_)
        # pca = PCA()
    
        
        voting.fit(train_X , train_y)

        acc , pre, rec , f1s , kap = f1_score_(val_X , val_y , voting)

        auc = roc_auc(val_y , voting.predict_proba(val_X))

        itr = itrr(acc)

        acc_s.append(acc)
        #pre_s.append(pre)
        #rec_s.append(rec)
        #f1s_s.append(f1s)
        kap_s.append(kap)
        auc_s.append(auc)
        itr_s.append(itr)

        print('[%.6f %.6f %.6f %.6f]' % (acc , kap , auc , itr) )

    print('%d %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f' % (freq , np.max(acc_s) , np.min(acc_s) , np.mean(acc_s) , np.var(acc_s) , np.max(kap_s) , np.min(kap_s) , np.mean(kap_s) , np.var(kap_s) ,np.max(auc_s) , np.min(auc_s) , np.mean(auc_s) , np.var(auc_s) ,np.max(itr_s) , np.min(itr_s) , np.mean(itr_s) , np.var(itr_s)) )

[0.880000 0.866667 0.894044 2.412176]
[0.740000 0.711111 0.811778 1.671001]
[0.800000 0.777778 0.909822 1.966015]
[0.660000 0.622222 0.866578 1.319335]
[0.860000 0.844444 0.951200 2.293900]
[0.780000 0.755556 0.902289 1.864377]
[0.920000 0.911111 0.944933 2.666155]
[0.780000 0.755556 0.897111 1.864377]
[0.660000 0.622222 0.826289 1.319335]
[0.820000 0.800000 0.879711 2.071265]
10 0.920000 0.660000 0.790000 0.006740 0.911111 0.622222 0.766667 0.008321 0.951200 0.811778 0.888376 0.001818 2.666155 1.319335 1.944794 0.174742
[0.820000 0.800000 0.911867 2.071265]
[0.840000 0.822222 0.929000 2.180431]
[0.920000 0.911111 0.969822 2.666155]
[0.840000 0.822222 0.925467 2.180431]
[0.880000 0.866667 0.921867 2.412176]
[0.800000 0.777778 0.892444 1.966015]
[0.920000 0.911111 0.943467 2.666155]
[0.840000 0.822222 0.921467 2.180431]
[0.840000 0.822222 0.927333 2.180431]
[0.880000 0.866667 0.920978 2.412176]
15 0.920000 0.800000 0.858000 0.001476 0.911111 0.777778 0.842222 0.001822 0.969822 0.892444 

In [ ]:
'''
for freq in [20]:
    for t in range(20): #每种频率随机测试 20 次
        #读取数据
        train_X_ , train_y , val_X_ , val_y = combine(freq=freq) #10 15 20 25 hz

        #提取特征
        train_X = feature_extraction_sub_band_power(train_X_)
        val_X = feature_extraction_sub_band_power(val_X_)

        #初始化所有分类器
        svc = SVC(probability=True)
        rf =  RandomForestClassifier()
        adaboost = AdaBoostClassifier()
        xgb = xgboost.XGBClassifier()
        lgbm = lightgbm.LGBMClassifier()
        gbc = GradientBoostingClassifier()
        knn = KNeighborsClassifier()
        dt = DecisionTreeClassifier()
        lda = LDA()
        nb = GaussianNB()
        mlp = MLPClassifier()

        classifiers = [xgb , lgbm , gbc , rf , svc , dt , lda , adaboost , mlp , nb , knn ]

        voting = VotingClassifier(estimators=[('xgboost' , xgb) , ('lightgbm' , lgbm) , ('gradientboosting' , gbc) ,
                                 ('randomforest' , rf) , ('svm' , svc) , ('decisiontree' , dt) , ('lda' , lda) ,
                                 ('adaboost' , adaboost) , ('mlp' , mlp) , ('naive bayes' , nb) , ('kneighbors' , knn)] ,
                    voting='soft' , n_jobs=-1)

        #特征选择
        chosen_idx = choose_common_feature_idx(train_X , train_y , classifiers=classifiers , use_ratio=0.25)

        #chosen_idx=np.array(list(range(train_X.shape[1])))

        train_X_chosen = train_X[: , chosen_idx]    

        voting.fit(train_X_chosen , train_y)    
        
        val_X_chosen = val_X[: , chosen_idx]

        con_mat_heatmap(val_X_chosen , val_y , voting , color=None ,
                        #png_path = 'v1_paper_pic/sub-band power/有特征选择/%shz/%s.png' % (freq , t+1) )
                        png_path = 'c://Users/qq122/Desktop/v1_paper_pic/sub-band power/有特征选择/%shz/%s.png' % (freq , t+1) )
                        '''